# 評価用動画の作成 

16_convertで変換した４つの走行モデルと, 21_convert_detect の環境認識モデルをつかって評価用の動画を作成します。

対応デバイス: Jetson Xavier NX, Jetson Orin Nano

In [1]:
import Jetson.GPIO as GPIO

BOARD_NAME = GPIO.gpio_pin_data.get_data()[0]

mode_descriptions = {
    "JETSON_NX": ["15W_2CORE", "15W_4CORE", "15W_6CORE", "10W_2CORE", "10W_4CORE"],
    "JETSON_XAVIER": ["MAXN", "MODE_10W", "MODE_15W", "MODE_30W"],
    "JETSON_NANO": ["MAXN", "5W"],
    "JETSON_ORIN": ["MAXN", "MODE_15W", "MODE_30W", "MODE_40W"],
    "JETSON_ORIN_NANO": ["MODE_15W", "MODE_7W"]
}

product_names = {
    "JETSON_NX": "Jetson Xavier NX",
    "JETSON_XAVIER": "Jetson AGX Xavier",
    "JETSON_NANO": "Jetson Nano",
    "JETSON_ORIN": "Jetson AGX Orin",
    "JETSON_ORIN_NANO": "Jetson Orin Nano"
}

# ボードごとのI2Cバス番号と初期Powerモードを定義する
board_settings = {
    "JETSON_NX": (8, 3),
    "JETSON_XAVIER": (8, 2),
    "JETSON_NANO": (1, 0),
    "JETSON_ORIN": (7, 0),
    "JETSON_ORIN_NANO": (7, 0)
}

i2c_busnum, power_mode = board_settings.get(BOARD_NAME, (None, None))
mode_description = mode_descriptions.get(BOARD_NAME, [])
product_name = product_names.get(BOARD_NAME, "未知のボード")

if power_mode is not None and power_mode < len(mode_description):
    mode_str = mode_description[power_mode]
    print("------------------------------------------------------------")
    print(f"{product_name}を認識: I2Cバス番号: {i2c_busnum}, Powerモード: {mode_str}({power_mode})に設定します。")
    print("------------------------------------------------------------")
else:
    print("未知のボードまたは不正なモードです。")

------------------------------------------------------------
Jetson Orin Nanoを認識: I2Cバス番号: 7, Powerモード: MODE_15W(0)に設定します。
------------------------------------------------------------


In [2]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のため電力モードは変更できません。")
else:
    !echo "jetson" | sudo -S nvpmodel -m $power_mode

Docker起動のため電力モードは変更できません。


In [3]:
!echo "jetson" | sudo -S nvpmodel -q

NVPM WARN: power mode is not set!


In [4]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のためjetson_clocksは起動できません。")
else:
    !echo "jetson" | sudo -S jetson_clocks

Docker起動のためjetson_clocksは起動できません。


In [5]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob
import datetime

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [6]:
LOAD_TASK = ['camera']
CATEGORIES = ["main","university","office","market","hotel","hospital","library","restraunt","etc"]
BASE_FPS = [30,60]

IMG_WIDTH = 224
IMG_HEIGHT = 224
current_path = os.getcwd()
SKIP = [1,2,3,4,5]

model_a_widget = ipywidgets.Dropdown(options=[],description='走行モデル')
model_a_time_widget = ipywidgets.Label(description='作成日時')
load_a_button = ipywidgets.Button(description='走行モデル読込み')
model_b_widget = ipywidgets.Dropdown(options=[],description='走行モデル')
model_b_time_widget = ipywidgets.Label(description='作成日時')
load_b_button = ipywidgets.Button(description='走行モデルを読込み')
model_c_widget = ipywidgets.Dropdown(options=[],description='走行モデル')
model_c_time_widget = ipywidgets.Label(description='作成日時')
load_c_button = ipywidgets.Button(description='走行モデル読込み')
model_d_widget = ipywidgets.Dropdown(options=[],description='走行モデル')
model_d_time_widget = ipywidgets.Label(description='作成日時')
load_d_button = ipywidgets.Button(description='走行モデル読込み')
model_e_widget = ipywidgets.Dropdown(options=[],description='環境モデル')
model_e_time_widget = ipywidgets.Label(description='作成日時')
load_e_button = ipywidgets.Button(description='環境モデル読込み')

load_datasets_widget0 = ipywidgets.Dropdown(options=[], description='dataset0')
load_task_widget0 = ipywidgets.Dropdown(options=LOAD_TASK, description='task')
load_datasets_widget1 = ipywidgets.Dropdown(options=[], description='dataset1')
load_task_widget1 = ipywidgets.Dropdown(options=LOAD_TASK, description='task')

camera_fps_widget = ipywidgets.Dropdown(options=BASE_FPS, description='camera_fps')

movie_button = ipywidgets.Button(description='動画の作成')
movie_name_widget = ipywidgets.Text(description='動画名',value="run_video_dual_merge_trt")
movie_skip_dropdown = ipywidgets.Dropdown(options=SKIP, description='skip(枚)', index=1)

building_dropbox = ipywidgets.Dropdown(options=CATEGORIES, description='Building')
speed_raw_fast_slider = ipywidgets.IntSlider(description='Speed fast', min=1, max=224, step=1, value=224, orientation='horizontal')
speed_raw_slow_slider = ipywidgets.IntSlider(description='Speed slow', min=1, max=224, step=1, value=120, orientation='horizontal')

In [7]:
def change_load_task(change):
    global current_path
    try:
        path = os.path.join(current_path,load_task_widget0.value)
        files = os.listdir(path)
        dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
        dirs = [f for f in files if f != ".ipynb_checkpoints"]
        dirs = sorted(dirs)
        load_datasets_widget0.options = dirs
        load_datasets_widget1.options = dirs
    except:
        write_log(path + "が存在していません。")
        load_datasets_widget0.options = []
        load_datasets_widget1.options = []
load_task_widget0.observe(change_load_task)
change_load_task(LOAD_TASK[0])

In [8]:
def load_model(widget, model_var_name):
    try:
        write_log(f"{widget.value}の読込を実行します(初回は時間がかかります)。")
        model = TRTModule()
        model.load_state_dict(torch.load(widget.value))
        write_log(f"{widget.value}の読込に成功しました。")
        globals()[model_var_name] = model  # 成功した場合、グローバル変数にモデルをセット
        load_flag_var_name = f"load_{model_var_name}"
        write_log(f"{load_flag_var_name}の読込に成功しました。")
        globals()[load_flag_var_name] = True  # 対応するloadフラグをTrueにセット
    except Exception as e:
        write_log(f"【Error】 {e} : {widget.value} の読込に失敗しました。")
        
# モデル読み込み関数を各ボタンのクリックイベントにバインドする例
load_a_button.on_click(lambda change: load_model(model_a_widget, 'model_a_trt'))
load_b_button.on_click(lambda change: load_model(model_b_widget, 'model_b_trt'))
load_c_button.on_click(lambda change: load_model(model_c_widget, 'model_c_trt'))
load_d_button.on_click(lambda change: load_model(model_d_widget, 'model_d_trt'))
load_e_button.on_click(lambda change: load_model(model_e_widget, 'model_e_trt'))

In [9]:
def model_list(type):
    try:
        files = glob.glob(type + '/*.pth', recursive=True)
        ts = os.path.getctime(files[0])
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        if type == "model_trt":
            model_a_widget.options = files
            model_a_time_widget.value = s
            model_b_widget.options = files
            model_b_time_widget.value = s
            model_c_widget.options = files
            model_c_time_widget.value = s
            model_d_widget.options = files
            model_d_time_widget.value = s
        elif type == "model_class_trt":
            model_e_widget.options = files
            model_e_time_widget.value = s
    except Exception as e:
        model_a_widget.options = []
        model_b_widget.options = []
        model_c_widget.options = []
        model_d_widget.options = []
        model_e_widget.options = []
        write_log(f"Error:{e}")
model_list("model_trt")
model_list("model_class_trt")

def update_model_time_widget(widget, time_widget):
    """指定されたウィジェットのファイル選択が変更された際の処理。ファイルの作成時間を表示ウィジェットに設定する。"""
    file = widget.value
    try:
        ts = os.path.getctime(file)
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        time_widget.value = s
    except Exception as e:
        time_widget.value = "Error: " + str(e)

# 各モデル選択ウィジェットの変更を監視し、対応する時刻表示ウィジェットを更新
model_a_widget.observe(lambda change: update_model_time_widget(model_a_widget, model_a_time_widget), names='value')
model_b_widget.observe(lambda change: update_model_time_widget(model_b_widget, model_b_time_widget), names='value')
model_c_widget.observe(lambda change: update_model_time_widget(model_c_widget, model_c_time_widget), names='value')
model_d_widget.observe(lambda change: update_model_time_widget(model_d_widget, model_d_time_widget), names='value')
model_e_widget.observe(lambda change: update_model_time_widget(model_e_widget, model_e_time_widget), names='value')

## ロジック

31_dual_camera_runからコピーしてもってくる

In [10]:
def get_model(status, target):
    if target == "main" or target == "university" or target == "office" or target == "restraunt":
        if status == STATUS_OUT_A:
            name = "model_a"
            return name,model_a_trt
        elif status == STATUS_OUT_B:
            name = "model_b"
            return name,model_b_trt
        elif status == STATUS_IN:
            name = "model_d"
            return name,model_d_trt
        else:
            name = "model_d"
            return name,model_d_trt 
    else:
        if status == STATUS_OUT_A:
            name = "model_a"
            return name,model_a_trt
        elif status == STATUS_OUT_B:
            name = "model_a"
            return name,model_a_trt
        elif status == STATUS_IN:
            name = "model_c"
            return name,model_c_trt
        else:
            name = "model_c"
            return name,model_c_trt 

In [11]:
CATEGORIES = ["main","university","office","market","hotel","hospital","library","restraunt","etc"]

def get_target(target):
    prebuilding = ""
    if target == "main":
        prebuilding = "restraunt"
    elif target == "university":
        prebuilding = "main"
    elif target == "office":
        prebuilding = "university"
    elif target == "restraunt":
        prebuilding = "office"
    elif target == "market":
        prebuilding = "office"
    elif target == "hotel":
        prebuilding = "market"
    elif target == "hospital":
        prebuilding = "hotel"
    elif target == "library":
        prebuilding = "hospital"
    
    return prebuilding

In [12]:
selected_color = ""
def change_color(color):
    global selected_color
    write_log(f"Change color:{color}")
    selected_color = color

In [13]:
def init_params():
    time_count = 0
    time_count_stop = 0

In [14]:
time_count = 0

def check_non_detection_period(elapsed_time_ms):
    global time_count
    time_count+=1
    emulation_time = ((1000/camera_fps_widget.value) * movie_skip_dropdown.value) * time_count
    write_log(f"emulation_time:{emulation_time}")
    if emulation_time > elapsed_time_ms:
        return True
    else:
        return False

def update_last_detect_time():
    global time_count
    time_count = 0

In [15]:
def get_stop_time(target):
    if target == "library" or target == "market":
        return 1000
    else:
        return 2000

In [16]:
time_count_stop = 0
def check_stop_time(current_time, target_detected_time, stop_time):
    global time_count_stop
    time_count_stop += 1
    emulation_time = (1000/camera_fps_widget.value) * movie_skip_dropdown.value * time_count_stop
    
    if emulation_time > stop_time:
        return True
    else:
        return False
    #if current_time - target_detected_time >= stop_time:
    #    return True:
    #else:
    #    return False:

In [17]:
def update_status_and_action(current_status, category_index, last_detect, detect_count, prebuilding, target, stop_time, target_detected_time=None):
    """
    現在の状態に基づき、次のアクションを決定し、状態を更新します。ターゲット発見後、指定された秒数でSTOPに遷移します。

    Args:
        current_status (int): 現在の状態
        category_index (int): 現在の認識結果のカテゴリインデックス
        last_detect (int): 前回の認識結果のカテゴリインデックス
        detect_count (int): 連続して認識された回数
        prebuilding (str): ターゲットの1つ前の建物名
        target (str): ターゲットとなる建物名
        stop_time (int): ターゲットを検出してから停止するまでの秒数
        target_detected_time (float, optional): ターゲットを最初に検出した時刻

    Returns:
        tuple: 更新された状態、連続認識回数、ターゲット検出時刻
    """
    global selected_model
    new_status = current_status
    new_detect_count = detect_count
    current_time = time.time()  # 現在時刻を取得

    # STATUS_TARGETに遷移した直後にタイムスタンプを記録
    if current_status == STATUS_TARGET and target_detected_time is None:
        target_detected_time = current_time

    # STATUS_TARGET状態で、指定された秒数が経過した場合にSTATUS_STOPに遷移
    if current_status == STATUS_TARGET:
        if check_stop_time(current_time, target_detected_time, stop_time):
            change_color("red")
            new_status = STATUS_STOP
            #PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)    
            target_detected_time = None  # タイムスタンプをリセット
            write_log("STOP!")
            

    if current_status == STATUS_OUT_A:
        if CATEGORIES[category_index] == target:
            if CATEGORIES[last_detect] == target:
                new_detect_count += 1
                if new_detect_count > 10:
                    write_log(f"Detect1 {target}")
                    change_color("yellow")
                    new_status = STATUS_OUT_B
                    new_detect_count = 0
                    update_last_detect_time()
            else:
                new_detect_count = 1
    elif current_status == STATUS_OUT_B:
        if check_non_detection_period(3000) == True:
            if CATEGORIES[category_index] == prebuilding:
                if CATEGORIES[last_detect] == prebuilding:
                    new_detect_count += 1
                    if new_detect_count > 5:
                        write_log(f"Detect2 {prebuilding}")
                        change_color("purple")
                        new_status = STATUS_IN
                        new_detect_count = 0
                else:
                    new_detect_count = 1
    elif current_status == STATUS_IN:
        if CATEGORIES[category_index] == target:
            if CATEGORIES[last_detect] == target:
                new_detect_count += 1
                if new_detect_count > 5:
                    change_color("orange")
                    write_log(f"Detect3 {target}")
                    new_status = STATUS_TARGET
                    new_detect_count = 0
            else:
                new_detect_count = 1
    return new_status, new_detect_count, target_detected_time

In [18]:
# 選択された色によって異なる色の円を描く
def draw_led(img0, selected_color):
    if selected_color == "white":
        color = (255, 255, 255) # BGRで白
    elif selected_color == "yellow":
        color = (0, 255, 255) # BGRで黄
    elif selected_color == "orange":
        # オレンジ色の正確なBGR値については調整が必要かもしれません
        color = (0, 165, 255) # BGRでオレンジ
    elif selected_color == "red":
        color = (0, 0, 255) # BGRで赤
    elif selected_color == "purple":
        color = (255, 0, 255) # BGRで紫
    else:
        color = (0, 0, 0) # その他の場合は黒

    # 3つの円を描く
    for i in range(3):
        img0 = cv2.circle(img0, (95 + i*15, 30), 5, color, -1) # -1で中を塗りつぶす

    return img0

In [19]:
import cv2
import glob
from utils import preprocess
import re
import torch.nn.functional as F
import torch
from utils import preprocess
import subprocess
import time
from torch2trt import TRTModule
import datetime
import torch.nn.functional as F

def extract_numbers(filename):
    matches = re.findall(r'(\d+)', filename)
    if matches and len(matches) >= 3: 
        return int(matches[-1])  
    else:
        return float('inf')

def get_file_names(path):
    file_names = os.listdir(path)
    file_names = [os.path.join(path, file_name) for file_name in file_names]
    image_names = []

    image_names = sorted(file_names, key=lambda f: extract_numbers(os.path.basename(f)))
    image_names = [f for f in image_names if os.path.splitext(f)[1].lower() == ".jpg"]
    
    return image_names

STATUS_IN = 1
STATUS_OUT_A = 2
STATUS_OUT_B = 3
STATUS_TARGET = 4
STATUS_STOP = 5
    
def make_movie(change):
    global model_a_trt,model_b_trt,model_c_trt,model_d_trt,model_e_trt,current_path, status, count

    if not movie_name_widget.value.strip():
        write_log("ファイル名を指定してください。")
        return 
    
    init_params()
    
    change_color("white")
    count = 0
    write_log("動画を作成します。")
    path = os.path.join(current_path,"video")
    if not os.path.exists(path):
        subprocess.call(['mkdir', '-p', path])
    output = path + "/" + movie_name_widget.value + ".mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(camera_fps_widget.value/movie_skip_dropdown.value)
    outfh = cv2.VideoWriter(output, fourcc, fps, (224*2, 224))
    
    xy_path0 = os.path.join(current_path, load_task_widget0.value, load_datasets_widget0.value, "xy")     
    file_list0 = get_file_names(xy_path0)
    xy_path1 = os.path.join(current_path, load_task_widget1.value, load_datasets_widget1.value, "xy")     
    file_list1 = get_file_names(xy_path1)
    detect_count = 0
    last_detect = -1
    status = STATUS_OUT_A
    lap = 0
    model = model_a_trt
    target = building_dropbox.value
    stop_time = get_stop_time(target)
    target_detected_time = None
    try:
        res_num = len(file_list0)
        write_log(f"画像枚数:{res_num}")
        count = 0
        skip_movie = movie_skip_dropdown.value
        last_lap_time = -5
        current_time = 0
        terminal_time = 1/(int(30/skip_movie))
        process_drive_time = 0
        process_detect_time = 0
        total_process_drive_time = 0
        total_process_detect_time = 0
        target_detect_time = 0
        stop_time = get_stop_time(target)
        for i, file_name in enumerate(file_list0):
            if i % skip_movie == 0:
                current_time += terminal_time
                img0 = cv2.imread(file_name)
                img1 = cv2.imread(file_list1[i])
                
                process_detect_time = time.time()
                # 画像の前処理
                img0_preprocessed = preprocess(img0).half()
                img1_preprocessed = preprocess(img1).half()
                
                process_drive_time = time.time()
                output = model(img0_preprocessed).detach().cpu().numpy().flatten()
                result_x = float(output[0])
                result_y = float(output[1])
                result_x = int(IMG_WIDTH * (result_x / 2.0 + 0.5))
                result_y = int(IMG_HEIGHT * (result_y / 2.0 + 0.5))    
                img0 = cv2.circle(img0, (int(result_x), int(result_y)), 8, (255, 0, 0), 3)

                # Speed
                result_speed = output[3]
                result_speed = int(IMG_WIDTH * (result_speed / 2.0 + 0.5))
                if result_speed > 224:
                    result_speed = 244
                elif result_speed < 0:
                    result_speed = 0
                total_process_drive_time += time.time() - process_drive_time 
                 
                output = model_e_trt(img1_preprocessed).detach()
                output = F.softmax(output, dim=1).cpu().numpy().flatten()
                category_index = output.argmax()
                total_process_detect_time += time.time() - process_detect_time 
                
                # ターゲットの1つ前の建物名を取得
                prebuilding = get_target(building_dropbox.value)
                # 環境認識後の処理（動画作成時も同じロジックを使う)
                status, detect_count, target_detected_time = update_status_and_action(status, category_index, last_detect, detect_count, prebuilding, target, stop_time, target_detected_time)
                last_detect = category_index
            
                model_name, model = get_model(status, target)                
                # 使用可能なモデル名
                str_models = ["model_a", "model_b", "model_c", "model_d"]
                # モデル名を画像に表示
                status_pos_y = 30  # 初期のY位置
                for str_model in str_models:
                    # 選択されているモデルは赤色、それ以外は白色で表示
                    if str_model == model_name:
                        color = (0, 0, 255)  # BGRで赤
                    else:
                        color = (255, 255, 255)  # BGRで白

                    # モデル名を画像に描画
                    img0 = cv2.putText(img0, str_model, (150, status_pos_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    status_pos_y += 20  # 次のモデル名のために位置を下に移動
                    
                # スピードを設定(statusによって速度を変える)
                if status == STATUS_OUT_A or status == STATUS_OUT_B:
                    speed = speed_raw_fast_slider.value
                elif status == STATUS_IN:
                    speed = speed_raw_slow_slider.value
                elif status == STATUS_TARGET:
                    speed = speed_raw_slow_slider.value
                    img0 = cv2.putText(img0,"time:"+str(stop_time),(10,140),cv2.FONT_HERSHEY_SIMPLEX,0.4,(255,255,255),2)
                elif status == STATUS_STOP:
                    #PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
                    speed = 0
                    img0 = cv2.putText(img0,"time:"+str(stop_time),(10,140),cv2.FONT_HERSHEY_SIMPLEX,0.4,(255,255,255),2)
                
                
                img0 = cv2.putText(img0,"speed:"+str(speed),(140,150),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
                
                
                # LED
                img0 = draw_led(img0, selected_color)
                    
                # 状態に対応するテキストと色を設定
                statuses = {
                    STATUS_OUT_A: ('OUT_A', (255, 255, 255)),
                    STATUS_OUT_B: ('OUT_B', (255, 255, 255)),
                    STATUS_IN:    ('IN',    (255, 255, 255)),
                    STATUS_TARGET:('TARGET',  (255, 255, 255)),
                    STATUS_STOP:  ('STOP',  (255, 255, 255)),
                }

                # 現在の状態の色を赤に設定
                statuses[status] = (statuses[status][0], (0, 0, 255))

                # ステータス表示位置の初期値
                status_pos_y = 30

                for key, (text, color) in statuses.items():
                    # ステータステキストを画像に描画
                    img0 = cv2.putText(img0, text, (10, status_pos_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    status_pos_y += 20  # 次のテキストのために位置を下に移動
                
                # 各カテゴリのスコアをパーセンテージで表示する改善版
                thickness = 2  # テキストの太さを細くする
                font_scale = 0.4  # フォントの大きさを小さく調整
                font_height = 15  # フォントの高さ（およその値）

                # 画像の高さを取得
                img_height = img1.shape[0]

                # 表示可能な最大カテゴリ数を計算
                max_categories = (img_height // font_height) - 2  # 上下に少し余裕を持たせる

                # 各カテゴリのスコアをパーセンテージで表示（表示可能数に制限）
                for h, score in enumerate(list(output)[:max_categories]):
                    text = f"{CATEGORIES[h]} : {score * 100:.1f}%"  # 小数点以下1桁まで表示
                    color = (0, 0, 255) if h == category_index else (255, 255, 255)  # 最も高いスコアのカテゴリを赤色で表示
                    position = (10, h * font_height + 20)  # テキストを描画する位置
                    img1 = cv2.putText(img1, text, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness)

                if len(CATEGORIES) > max_categories:
                    # 画像に収まりきらないカテゴリがある場合の処理
                    overflow_text = f"...and more"
                    img1 = cv2.putText(img1, overflow_text, (10, (max_categories + 1) * font_height + 20), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness)

                if i%(skip_movie*10) == 0:
                    write_log(f"{current_time:.1f}秒まで完了, 走行推論: {total_process_drive_time/10*1000:.1f}ms, 環境推論: {total_process_detect_time/10*1000:.1f}ms, {int(i/skip_movie)}枚目/{int(res_num/skip_movie)}枚中を処理中")
                    total_process_detect_time = 0
                    total_process_drive_time = 0
                
                img = cv2.hconcat([img0, img1])  # imgは224x448サイズになります
        
                outfh.write(img)
                del img
    except Exception as e:
        write_log(f"Error: {e}")
    finally:
        # エラーが発生しても確実にリソースを解放する
        outfh.release()
        write_log("動画の出力が完了しました。")

movie_button.on_click(make_movie)

In [20]:
separator = ipywidgets.HTML('<hr style="border-color:gray;margin:10px 0"/>')
title1 = ipywidgets.HTML('<b>【1.走行モデル(外回り直進)の選択】</b> 走行モデルを選択してください。')
title2 = ipywidgets.HTML('<b>【2.走行モデル(外回り右折)の選択】</b> 走行モデルを選択してください。')
title3 = ipywidgets.HTML('<b>【3.走行モデル(内回り直進)の選択】</b> 走行モデルを選択してください。')
title4 = ipywidgets.HTML('<b>【4.走行モデル(内回り右折)の選択】</b> 走行モデルを選択してください。')
title5 = ipywidgets.HTML('<b>【5.環境モデルの選択】</b> 環境モデルを選択してください。')
title6 = ipywidgets.HTML('<b>【6.目標】</b> 目標物を選択してください。')
title7 = ipywidgets.HTML('<b>【7.データセットの選択】</b> データセットを選択してください。')
title8 = ipywidgets.HTML('<b>【8.走行動画の作成】</b> 動画ファイル名を指定してください。')

convert_widget = ipywidgets.VBox([
    separator,
    title1,
    ipywidgets.HBox([model_a_widget,model_a_time_widget,load_a_button]),
    process_widget,
    title2,
    ipywidgets.HBox([model_b_widget,model_b_time_widget,load_b_button]),
    process_widget,
    title3,
    ipywidgets.HBox([model_c_widget,model_c_time_widget,load_c_button]),
    process_widget,
    title4,
    ipywidgets.HBox([model_d_widget,model_d_time_widget,load_d_button]),
    process_widget,
    title5,
    ipywidgets.HBox([model_e_widget,model_e_time_widget,load_e_button]),
    process_widget,
    title6,
    building_dropbox,
    speed_raw_fast_slider,
    speed_raw_slow_slider,
    title7,
    ipywidgets.HBox([load_datasets_widget0,load_task_widget0]),
    ipywidgets.HBox([load_datasets_widget1,load_task_widget1]),
    process_widget,
    title8,
    camera_fps_widget,
    ipywidgets.HBox([movie_name_widget,movie_skip_dropdown,movie_button]),
    process_widget,
])
display(convert_widget)